In [1]:
import asyncio
import traceback
import sounddevice as sd
from scipy.io.wavfile import write
import base64
import os
from dotenv import load_dotenv

from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

load_dotenv()

# Get the HumeStreamClient key from the environment variable
hume_stream_client_key = os.getenv("HUME_STREAM_CLIENT_KEY")

# Record audio
samplerate = 44100  # Hertz
duration = 4  # seconds
filename = 'output.wav'
print(f"Recording for {duration} seconds...")
myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, blocking=True)
print("Recording complete. Saving the audio as output.wav")
write(filename, samplerate, myrecording)

# Function to encode audio (base64 encoding)
def encode_audio(filename):
    with open(filename, 'rb') as audio_file:
        return base64.b64encode(audio_file.read())

# Hume API interaction
async def main():
    try:
        client = HumeStreamClient(hume_stream_client_key)
        burst_config = BurstConfig()
        prosody_config = ProsodyConfig()
        async with client.connect([burst_config, prosody_config]) as socket:
            encoded_audio = encode_audio(filename)
            await socket.reset_stream()
            result = await socket.send_bytes(encoded_audio)
            print("Received response from Hume")
            return result  # return the result
    except Exception:
        print(traceback.format_exc())

# This line is specific to IPython/Jupyter Notebook:
result = await main()  # 'await' can be used directly in a cell in Jupyter Notebook

print(result)


Recording for 4 seconds...
Recording complete. Saving the audio as output.wav
Traceback (most recent call last):
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/hume/_stream/hume_stream_client.py", line 93, in connect
    async with websockets.connect(  # type: ignore[attr-defined]
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/client.py", line 629, in __aenter__
    return await self
           ^^^^^^^^^^
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/client.py", line 647, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/client.py", line 654, in __await_impl__
    await protocol.handshake(
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/client.py", line 

In [ ]:

# find the emotion ids from emotion names
# emotion_id = []
# for  i, e in enumerate(emotions):
#     if e['name'] in emotion_names:
#         emotion_id.append(i)
# print(emotion_id)

In [4]:
emotions = result['prosody']['predictions'][0]['emotions']
emotion_names = ['Anger', 'Calmness', 'Embarrassment', 'Excitement', 'Romance', 'Sadness']
emotion_colors = ['red',   'green',     'yellow',        'orange',      'pink',     'blue']
emotion_id = [4, 9, 22, 26, 38, 39]
emotion_values = [emotions[i]['score'] for i in emotion_id]
print(emotion_values)


[0.008292632177472115, 0.07689502835273743, 0.012177638709545135, 0.14934971928596497, 0.019728396087884903, 0.025075571611523628]


In [5]:
emotion_values

[0.008292632177472115,
 0.07689502835273743,
 0.012177638709545135,
 0.14934971928596497,
 0.019728396087884903,
 0.025075571611523628]